<a href="https://colab.research.google.com/github/Snowdenstyll/Lottery/blob/main/server/jupyterNotebooks/Keno_Lottery_Claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Data Preprocessing
def prepare_data(csv_file):
    # Read the data
    df = pd.read_csv(csv_file)

    # Separate A and P games
    df_A = df[df['AP'] == 'A'].drop(['PlayDate', 'AP'], axis=1)
    df_P = df[df['AP'] == 'P'].drop(['PlayDate', 'AP'], axis=1)

    # Process each game type separately
    def process_game_data(game_df):
        # Convert to float
        game_df = game_df.astype(float)

        # Normalize numbers to [0,1] range
        scaler = MinMaxScaler(feature_range=(0, 1))
        transformed_data = scaler.fit_transform(game_df.values)
        transformed_df = pd.DataFrame(data=transformed_data, columns=game_df.columns)

        # Create sequences
        number_of_rows = len(transformed_df)
        window_length = 5
        number_of_features = transformed_df.shape[1]

        X = np.empty([number_of_rows - window_length, window_length, number_of_features], dtype=float)
        y = np.empty([number_of_rows - window_length, number_of_features], dtype=float)

        for i in range(0, number_of_rows - window_length):
            X[i] = transformed_df.iloc[i:i+window_length, :]
            y[i] = transformed_df.iloc[i+window_length:i+window_length+1, :].values

        return X, y, scaler

    return process_game_data(df_A), process_game_data(df_P)

# Create the model
def create_model(window_length, number_of_features):
    model = Sequential([
        Input(shape=(window_length, number_of_features)),  # Explicit input layer
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(LSTM(64)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(number_of_features, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                 loss='mse',
                 metrics=['mae'])
    return model

# Train separate models for A and P games
def train_model(X, y, game_type):
    try:
        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Create and train the model
        model = create_model(X.shape[1], X.shape[2])

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
            ModelCheckpoint(f'best_model_{game_type}.keras',  # Changed extension to .keras
                          save_best_only=True,
                          monitor='val_loss')
        ]

        history = model.fit(X_train, y_train,
                          epochs=200,
                          batch_size=32,
                          validation_split=0.2,
                          callbacks=callbacks,
                          verbose=1)

        return model, history
    except Exception as e:
        print(f"Error training model for game type {game_type}: {str(e)}")
        raise

# Function to convert predictions back to actual numbers
def process_predictions(predictions, scaler, original_features_count=20):
    try:
        # Inverse transform the normalized predictions
        predictions_original = scaler.inverse_transform(predictions)

        # Round to nearest integers and ensure unique numbers
        rounded_predictions = []
        for pred in predictions_original:
            # Sort numbers and round them
            sorted_numbers = np.sort(np.round(pred))
            # Ensure no duplicates and numbers are within range
            unique_numbers = np.unique(np.clip(sorted_numbers, 1, 69))
            # If we don't have exactly 20 numbers, adjust
            while len(unique_numbers) < original_features_count:
                # Add missing numbers
                available_numbers = set(range(1, 70)) - set(unique_numbers)
                unique_numbers = np.append(unique_numbers, np.random.choice(list(available_numbers)))
                unique_numbers = np.sort(unique_numbers)
            rounded_predictions.append(unique_numbers[:20])

        return rounded_predictions
    except Exception as e:
        print(f"Error processing predictions: {str(e)}")
        raise

# Main execution
if __name__ == "__main__":
    try:
        # Process both game types
        print("Processing data...")
        (X_A, y_A, scaler_A), (X_P, y_P, scaler_P) = prepare_data("2024.csv")

        print("Training model A...")
        # Train models for both game types
        model_A, history_A = train_model(X_A, y_A, 'A')

        print("Training model P...")
        model_P, history_P = train_model(X_P, y_P, 'P')

        # Make predictions for next games
        print("Making predictions...")
        last_sequence_A = X_A[-1:]
        last_sequence_P = X_P[-1:]

        pred_A = model_A.predict(last_sequence_A)
        pred_P = model_P.predict(last_sequence_P)

        # Convert predictions to actual numbers
        numbers_A = process_predictions(pred_A, scaler_A)
        numbers_P = process_predictions(pred_P, scaler_P)

        print("\nPredicted numbers for next 'A' game:", numbers_A[0])
        print("Predicted numbers for next 'P' game:", numbers_P[0])

        # Evaluate models
        print("\nModel A Evaluation:")
        loss_A = model_A.evaluate(X_A, y_A)
        print(f"MSE: {loss_A[0]:.4f}, MAE: {loss_A[1]:.4f}")

        print("\nModel P Evaluation:")
        loss_P = model_P.evaluate(X_P, y_P)
        print(f"MSE: {loss_P[0]:.4f}, MAE: {loss_P[1]:.4f}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

Processing data...
Training model A...
Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 157ms/step - loss: 0.0608 - mae: 0.2062 - val_loss: 0.0496 - val_mae: 0.1849
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0366 - mae: 0.1558 - val_loss: 0.0475 - val_mae: 0.1605
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0345 - mae: 0.1451 - val_loss: 0.0440 - val_mae: 0.1609
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0354 - mae: 0.1506 - val_loss: 0.0441 - val_mae: 0.1658
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0332 - mae: 0.1473 - val_loss: 0.0436 - val_mae: 0.1594
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0356 - mae: 0.1486 - val_loss: 0.0436 - val_mae: 0.1587
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0343 - mae: 0.1469 - val_loss: 0.0434 - val_mae: 0.1598
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0326 - mae: 0.1447 - val_loss: 0.0433 - val_mae: 0.1611
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

def analyze_predictions(model, X, scaler, game_type, actual_data):
    # Get predictions
    predictions = model.predict(X)
    predictions = process_predictions(predictions, scaler)

    # Convert predictions to integers
    predictions = np.array(predictions).astype(int)

    # 1. Show next game prediction
    print(f"\nPredicted numbers for next {game_type} game:")
    print(sorted(predictions[-1]))

    # 1.1 Get top 10 and top 5 predicted numbers
    all_numbers = predictions.flatten()
    number_freq = Counter(all_numbers)

    top_10_numbers = [number for number, freq in number_freq.most_common(10)]
    top_5_numbers = top_10_numbers[:5]  # Top 5 are the first 5 of top 10

    print(f"\nTop 10 most frequently predicted numbers ({game_type} Game):", top_10_numbers)
    print(f"\nTop 5 most frequently predicted numbers ({game_type} Game):", top_5_numbers)

    # 2. Visualization of prediction accuracy
    plt.figure(figsize=(15, 5))

    # 2.1 Prediction vs Actual heatmap
    plt.subplot(1, 3, 1)
    actual = scaler.inverse_transform(actual_data[-1:])
    actual = np.round(actual).astype(int)[0]

    comparison_data = pd.DataFrame({
        'Predicted': sorted(predictions[-1]),
        'Actual': sorted(actual)
    })

    sns.heatmap(comparison_data.corr(), annot=True, cmap='coolwarm')
    plt.title(f'Prediction vs Actual Correlation ({game_type} Game)')

    # 2.2 Number frequency distribution
    plt.subplot(1, 3, 2)
    all_numbers = predictions.flatten()
    number_freq = Counter(all_numbers)

    plt.bar(number_freq.keys(), number_freq.values())
    plt.title(f'Number Frequency in Predictions ({game_type} Game)')
    plt.xlabel('Number')
    plt.ylabel('Frequency')

    # 2.3 Error distribution
    plt.subplot(1, 3, 3)
    errors = np.abs(predictions - actual).flatten()
    plt.hist(errors, bins=20)
    plt.title(f'Prediction Error Distribution ({game_type} Game)')
    plt.xlabel('Absolute Error')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

    # 3. Most frequently predicted numbers
    top_numbers = pd.DataFrame.from_dict(number_freq, orient='index', columns=['frequency'])
    top_numbers = top_numbers.sort_values('frequency', ascending=False).head(10)
    print(f"\nTop 10 most frequently predicted numbers ({game_type} Game):")
    print(top_numbers)



    return predictions

# Call this function for both models
#analyze_predictions(model_A, X_A, scaler_A, 'A', y_A)
#analyze_predictions(model_P, X_P, scaler_P, 'P', y_P)

In [44]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def create_features(df):
    """Create additional features for the model"""
    features = pd.DataFrame()

    # Basic statistics of previous numbers
    features['mean'] = df.mean(axis=1)
    features['std'] = df.std(axis=1)
    features['median'] = df.median(axis=1)

    # Number range features
    features['max_diff'] = df.max(axis=1) - df.min(axis=1)
    features['range_1_20'] = df.apply(lambda x: sum(1 for n in x if n <= 20), axis=1)
    features['range_21_40'] = df.apply(lambda x: sum(1 for n in x if 20 < n <= 40), axis=1)
    features['range_41_60'] = df.apply(lambda x: sum(1 for n in x if 40 < n <= 60), axis=1)
    features['range_61_plus'] = df.apply(lambda x: sum(1 for n in x if n > 60), axis=1)

    # Consecutive numbers
    features['consecutive_count'] = df.apply(lambda x: sum(1 for i in range(len(x)-1) if x.iloc[i+1] - x.iloc[i] == 1), axis=1)

    # Even/Odd ratio
    features['even_count'] = df.apply(lambda x: sum(1 for n in x if n % 2 == 0), axis=1)
    features['odd_count'] = df.apply(lambda x: sum(1 for n in x if n % 2 != 0), axis=1)

    return features

################

################

def prepare_improved_data(csv_file):
    df = pd.read_csv(csv_file)

    # Separate A and P games
    df_A = df[df['AP'] == 'A'].drop(['PlayDate', 'AP'], axis=1)
    df_P = df[df['AP'] == 'P'].drop(['PlayDate', 'AP'], axis=1)

    def process_game_data(game_df):
        # Convert to float
        game_df = game_df.astype(float)

        # Normalize numbers to [0,1] range
        scaler = MinMaxScaler(feature_range=(0, 1))
        transformed_data = scaler.fit_transform(game_df.values)
        transformed_df = pd.DataFrame(data=transformed_data, columns=game_df.columns)

        # Create sequences
        number_of_rows = len(transformed_df)
        window_length = 5
        number_of_features = transformed_df.shape[1]

        X = np.empty([number_of_rows - window_length, window_length, number_of_features], dtype=float)
        y = np.empty([number_of_rows - window_length, number_of_features], dtype=float)

        for i in range(0, number_of_rows - window_length):
            X[i] = transformed_df.iloc[i:i+window_length, :]
            y[i] = transformed_df.iloc[i+window_length:i+window_length+1, :].values

        return X, y, scaler

    return process_game_data(df_A), process_game_data(df_P)

def create_improved_model(window_length, number_of_features, output_features):
    model = Sequential([
        Input(shape=(window_length, number_of_features)),

        # CNN layers for feature extraction
        Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
        MaxPooling1D(pool_size=2),

        # LSTM layers
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.4),

        Bidirectional(LSTM(64, return_sequences=True)),
        Dropout(0.4),

        Bidirectional(LSTM(32)),
        Dropout(0.4),

        # Dense layers
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dropout(0.4),

        # Output layer
        Dense(output_features, activation='sigmoid')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    return model

def train_improved_model(X, y, game_type):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = create_improved_model(X.shape[1], X.shape[2], y.shape[1])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
        ModelCheckpoint(f'improved_model_{game_type}.keras', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
    ]

    history = model.fit(
        X_train, y_train,
        epochs=300,
        batch_size=32,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )

    return model, history

# Main execution for improved model
if __name__ == "__main__":
    print("Processing data with improved features...")
    (X_A, y_A, scaler_A), (X_P, y_P, scaler_P) = prepare_improved_data("2024.csv")

    print("Training improved model A...")
    model_A, history_A = train_improved_model(X_A, y_A, 'A')

    print("Training improved model P...")
    model_P, history_P = train_improved_model(X_P, y_P, 'P')

    # Make predictions and analyze results
    #print("Analyzing results...")

    #analyze_predictions(model_A, X_A, scaler_A, 'A', y_A)
    #analyze_predictions(model_P, X_P, scaler_P, 'P', y_P)

Processing data with improved features...
Training improved model A...
Epoch 1/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 190ms/step - loss: 0.0670 - mae: 0.2183 - val_loss: 0.0701 - val_mae: 0.2258 - learning_rate: 0.0010
Epoch 2/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0610 - mae: 0.2074 - val_loss: 0.0542 - val_mae: 0.1947 - learning_rate: 0.0010
Epoch 3/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0430 - mae: 0.1716 - val_loss: 0.0465 - val_mae: 0.1617 - learning_rate: 0.0010
Epoch 4/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0394 - mae: 0.1578 - val_loss: 0.0440 - val_mae: 0.1650 - learning_rate: 0.0010
Epoch 5/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0367 - mae: 0.1554 - val_loss: 0.0447 - val_mae: 0.1693 - learning_rate: 0.0010
Epoch 6/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0358 - mae: 0.1539 - val_loss: 0.0437 - val_mae: 0.1608 - learning_rate: 0.0010
Epoch 7/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0353 - mae: 0.1478 - val_los

In [45]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Make predictions and analyze results
print("Analyzing results...")

def make_predictions(model, X, scaler, game_type, actual_data):
    # Get predictions
    predictions = model.predict(X)
    predictions = process_predictions(predictions, scaler)

    # Convert predictions to integers
    predictions = np.array(predictions).astype(int)

    # 1. Show next game prediction
    print(f"\nPredicted numbers for next {game_type} game:")
    print(sorted(predictions[-1]))

    # For each prediction, we will select the top 10 numbers with the highest confidence
    # `predictions[-1]` is the prediction for the next game
    prediction_scores = predictions[-1]

    # Get the indices of the top 10 predictions with the highest confidence
    top_10_indices = np.argsort(prediction_scores)[::-1][:10]

    # Retrieve the top 10 predicted numbers using these indices
    top_10_predictions = prediction_scores[top_10_indices]

    # Convert the top 10 predictions to integers
    top_10_predictions = np.array(top_10_predictions).astype(int)

    # 1. Show the top 10 predictions based on confidence
    print(f"\nTop 10 predicted numbers with the most confidence for next {game_type} game:")
    print(sorted(top_10_predictions))


print("######################")
make_predictions(model_A, X_A, scaler_A, 'A', y_A)
make_predictions(model_P, X_P, scaler_P, 'P', y_P)

#analyze_predictions(model_A, X_A, scaler_A, 'A', y_A)
#analyze_predictions(model_P, X_P, scaler_P, 'P', y_P)
print("\n")

Analyzing results...
######################
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step

Predicted numbers for next A game:
[3, 7, 10, 14, 17, 20, 24, 27, 31, 33, 36, 40, 44, 47, 50, 54, 57, 61, 64, 67]

Top 10 predicted numbers with the most confidence for next A game:
[36, 40, 44, 47, 50, 54, 57, 61, 64, 67]
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step

Predicted numbers for next P game:
[4, 8, 10, 14, 17, 21, 25, 29, 32, 35, 38, 41, 44, 47, 51, 54, 57, 60, 63, 67]

Top 10 predicted numbers with the most confidence for next P game:
[38, 41, 44, 47, 51, 54, 57, 60, 63, 67]


